In [ ]:
# =========================================
# Cell 1 — Smart Setup (Sheets + Auth)
# =========================================

!pip -q install --upgrade \
  gspread==6.1.4 gspread-formatting==1.2.0 google-auth==2.38.0 google-auth-oauthlib==1.2.1 \
  pandas==2.2.2 requests==2.32.4 beautifulsoup4==4.12.3 lxml==5.3.0 reportlab==4.2.5 matplotlib==3.9.2

import os, re, json, math, time, datetime as dt
import pandas as pd, gspread
from gspread.exceptions import WorksheetNotFound
from google.oauth2.service_account import Credentials as SA_Creds
from gspread_formatting import set_frozen, CellFormat, Color, format_cell_ranges
try:
    from google.colab import auth as colab_auth
    IN_COLAB = True
except Exception:
    colab_auth = None
    IN_COLAB = False

KSA_TZ = dt.timezone(dt.timedelta(hours=3))
SHEET_ID = "1d2js0tZAIUzmVnKwlBHjr3NEvCLfsl6urEhKScXyPME"
SHEET_TAB = "Sheet1"

SCHEMA = [
    "اسم المنافسة",
    "الرقم المرجعي",
    "الجهة",
    "قيمة المنافسة",
    "اخر موعد للاستفسار",
    "اخر موعد للتقديم",
    "الرابط",
    "تاريخ_الإدراج"
]

pd.options.display.max_colwidth = 200
pd.options.display.width = 200

def today_ksa_date():
    return dt.datetime.now(KSA_TZ).date().isoformat()

def _first_existing(paths):
    for p in paths:
        if p and os.path.exists(p):
            return p
    return None

def _try_service_account():
    sa_path = _first_existing([
        "/content/etimad_service_account.json",
        "/content/drive/MyDrive/etimad_service_account.json",
        "/workspace/etimad_service_account.json",
        os.getenv("GSHEET_SA_JSON","")
    ])
    if not sa_path:
        return None
    creds = SA_Creds.from_service_account_file(
        sa_path,
        scopes=["https://www.googleapis.com/auth/spreadsheets",
                "https://www.googleapis.com/auth/drive"]
    )
    return gspread.authorize(creds)

def _try_colab_adc():
    if not IN_COLAB:
        return None
    try:
        colab_auth.authenticate_user()
        import google.auth
        from google.auth.transport.requests import Request
        creds, _ = google.auth.default(scopes=[
            "https://www.googleapis.com/auth/spreadsheets",
            "https://www.googleapis.com/auth/drive"
        ])
        if hasattr(creds, "expired") and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        return gspread.authorize(creds)
    except Exception:
        return None

def _try_oauth_file():
    cred_path = _first_existing([
        "/content/credentials.json",
        "/content/drive/MyDrive/credentials.json",
        os.getenv("GSHEET_OAUTH_JSON","")
    ])
    if not cred_path:
        return None
    return gspread.oauth(
        scopes=[
            "https://www.googleapis.com/auth/spreadsheets",
            "https://www.googleapis.com/auth/drive"
        ],
        credentials_filename=cred_path,
        authorized_user_filename="/content/authorized_user.json",
    )

def get_gspread_client():
    for fn in (_try_service_account, _try_colab_adc, _try_oauth_file):
        cli = fn()
        if cli is not None:
            return cli
    raise RuntimeError("No Google credentials available. Provide a Service Account JSON or credentials.json, or run in Colab and grant access.")

def open_or_create_worksheet(gc, sheet_id: str, tab_name: str):
    sh = gc.open_by_key(sheet_id)
    try:
        ws = sh.worksheet(tab_name)
    except WorksheetNotFound:
        ws = sh.add_worksheet(title=tab_name, rows=1000, cols=max(8, len(SCHEMA)))
    return ws

def ensure_headers(ws):
    existing = ws.row_values(1)
    if existing != SCHEMA:
        if ws.col_count < len(SCHEMA):
            ws.resize(rows=ws.row_count, cols=len(SCHEMA))
        ws.update('A1', [SCHEMA])
    set_frozen(ws, rows=1, cols=0)
    header_fmt = CellFormat(backgroundColor=Color(0.85, 0.93, 0.98))
    format_cell_ranges(ws, [('A1:{}1'.format(chr(ord('A')+len(SCHEMA)-1)), header_fmt)])

def sanity_print(ws):
    print(f"Connected to sheet: https://docs.google.com/spreadsheets/d/{SHEET_ID}")
    print(f"Worksheet: {SHEET_TAB}")
    print(f"Columns (A1 headers): {ws.row_values(1)}")
    print("Today (KSA):", today_ksa_date())

gc = get_gspread_client()
ws = open_or_create_worksheet(gc, SHEET_ID, SHEET_TAB)
ensure_headers(ws)
sanity_print(ws)
print("✅ Setup done")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.9/147.9 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 63.8 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# =========================================
# Cell 2 — Sheet utilities (today tenders , cleanup, highlight) — FIX v2
# =========================================

import datetime as _dt
from gspread_formatting import CellFormat, Color, format_cell_ranges

DATE_FMT = "%Y-%m-%d"

def _coerce_date_series(s):
    s = pd.to_datetime(s, errors="coerce")
    s = s.dt.strftime(DATE_FMT).fillna("")
    return s

def _coerce_value_series(s):
    s = s.astype(str).str.replace(r"[^0-9.]", "", regex=True)
    s = s.replace("", pd.NA)
    s = pd.to_numeric(s, errors="coerce")
    return s

def _ensure_schema_columns(df: pd.DataFrame) -> pd.DataFrame:
    for col in SCHEMA:
        if col not in df.columns:
            df[col] = pd.NA
    return df[SCHEMA].copy()

def _clear_body(ws):
    if ws.row_count > 1:
        ws.batch_clear([f"A2:{chr(ord('A')+len(SCHEMA)-1)}{ws.row_count}"])

def _append_rows(ws, rows):
    if not rows:
        return
    ws.append_rows(rows, value_input_option="RAW", table_range="A1")

def _read_all_as_df(ws) -> pd.DataFrame:
    vals = ws.get_all_values()
    if not vals:
        return pd.DataFrame(columns=SCHEMA)
    header = vals[0]
    data = vals[1:]
    df = pd.DataFrame(data, columns=header)
    for col in SCHEMA:
        if col not in df.columns:
            df[col] = pd.NA
    return df[SCHEMA].copy()

def weekly_cleanup(ws, today_str: str):
    df = _read_all_as_df(ws)
    if df.empty:
        return
    dates = pd.to_datetime(df["تاريخ_الإدراج"], errors="coerce")
    cutoff = pd.to_datetime(today_str) - pd.Timedelta(days=7)
    keep_mask = dates.isna() | (dates >= cutoff)
    new_df = df[keep_mask].copy()
    _clear_body(ws)
    rows = new_df.fillna("").values.tolist()
    _append_rows(ws, rows)

def highlight_today_rows(ws, today_str: str):
    last_row = ws.row_count
    if last_row < 2:
        return
    rng = f"A2:{chr(ord('A')+len(SCHEMA)-1)}{last_row}"
    fmt = CellFormat(backgroundColor=Color(0.98, 0.98, 0.90))
    format_cell_ranges(ws, [(rng, fmt)])

def write_today_only(ws, df_today: pd.DataFrame, today_str: str):
    df_today = df_today.copy()
    if "قيمة المنافسة" in df_today.columns:
        df_today["قيمة المنافسة"] = _coerce_value_series(df_today["قيمة المنافسة"])
    for c in ["اخر موعد للاستفسار", "اخر موعد للتقديم"]:
        if c in df_today.columns:
            df_today[c] = _coerce_date_series(df_today[c])
    df_today["تاريخ_الإدراج"] = today_str
    df_today = _ensure_schema_columns(df_today).fillna("")
    _clear_body(ws)
    _append_rows(ws, df_today.values.tolist())
    highlight_today_rows(ws, today_str)

def sync_today_sheet(df_today: pd.DataFrame):
    today_str = today_ksa_date()
    weekly_cleanup(ws, today_str)
    write_today_only(ws, df_today, today_str)
    print(f"Synced today-only rows: {len(df_today)}")
    print("Highlight applied.")
    print("Weekly cleanup done (older than 7 days).")


In [ ]:
# =========================================
# Cell 3 — Etimad scraper (requests list-cards → fields; keyword filter; fallback)
# =========================================

import re, time, random
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urljoin

LIST_PAGES = [
    "https://tenders.etimad.sa/Tender/AllTendersForVisitor",
    "https://portal.etimad.sa/Tender/AllTendersForVisitor",
]
UA_POOL = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0 Safari/537.36",
]

AR_KW = [
    "هاكاثون","هاكثون","الهاكاثون","ثون","ثونات","الهاكاثونات","هاكثونات","هاكاثونات",
    "ابتكار","يبتكرون","الابتكار","مبتكرون","نبتكر","ابتكارات","المبتكرون","ابتكاري",
    "فعاليات","فعالية","فعاليه","الفعاليات","الفعالية","الفعاليه",
]
EN_KW = ["hackathon","innovation","innovate","innovator","innovators","event","events","activity","activities"]

_AR_DIAC = re.compile(r"[\u0617-\u061A\u064B-\u0652\u0670\u065F]")
def _norm_ar(s):
    s = str(s or "")
    s = _AR_DIAC.sub("", s)
    s = s.replace("أ","ا").replace("إ","ا").replace("آ","ا")
    s = s.replace("ى","ي").replace("ئ","ي").replace("ؤ","و").replace("ة","ه")
    s = re.sub(r"\s+", " ", s).strip()
    return s
def _norm_all(s): return _norm_ar(s).lower()
def _kw_ok(name, entity):
    t = _norm_all((name or "") + " " + (entity or ""))
    for k in AR_KW:
        if _norm_ar(k) in t: return True
    for k in EN_KW:
        if k.lower() in t: return True
    return False

def _session():
    s = requests.Session()
    s.headers.update({
        "User-Agent": random.choice(UA_POOL),
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
        "Accept-Language": "ar-SA,ar;q=0.9,en;q=0.8",
        "Connection": "keep-alive",
    })
    return s
def _get(s, url, params=None, sleep=0.7, timeout=45):
    r = s.get(url, params=params, timeout=timeout, allow_redirects=True)
    r.raise_for_status()
    if sleep: time.sleep(sleep)
    return r

def _parse_date_iso(text):
    txt = str(text or "")
    m = re.search(r"(\d{4})[/-](\d{2})[/-](\d{2})", txt)
    if m: return f"{m.group(1)}-{m.group(2)}-{m.group(3)}"
    m = re.search(r"(\d{1,2})[/-](\d{1,2})[/-](\d{2,4})", txt)
    if m:
        d,mn,yy = m.groups(); yy = "20"+yy if len(yy)==2 else yy
        return f"{int(yy):04d}-{int(mn):02d}-{int(d):02d}"
    return ""

def _parse_money(text):
    if text is None: return None
    t = re.sub(r"[^\d.]", "", str(text))
    return None if t=="" else float(t)

def _closest_card(a):
    node = a
    for _ in range(6):
        if not node: break
        if node.name in ("div","section","article") and ("card" in " ".join(node.get("class", [])) or "media" in " ".join(node.get("class", []))):
            return node
        node = node.parent
    return a.find_parent(["div","section","article"]) or a.parent

def _extract_from_card(base_url, card, details_href):
    name = ""
    for sel in ["a.tender-title","a.title","h1","h2","h3",".tender-title",".title","a"]:
        el = card.select_one(sel)
        if el and el.get_text(strip=True) and "تفاصيل" not in el.get_text(strip=True):
            name = el.get_text(" ", strip=True)
            break
    if not name:
        name = card.get_text(" ", strip=True).split("\n")[0].strip()
    text = card.get_text(" ", strip=True)
    ref = ""
    m = re.search(r"(?:الرقم\s*المرجعي|Reference|Tender\s*No)\s*[:\-]?\s*([A-Za-z0-9\/\-]{6,})", text)
    if m: ref = m.group(1).strip()
    entity = ""
    m = re.search(r"(?:الجهة|Entity|الجهة\s*الحكومية)\s*[:\-]?\s*([^\|]+?)(?:\s{2,}|\|$)", text)
    if m: entity = m.group(1).strip()
    value = None
    m = re.search(r"(?:قيمة\s*وثائق\s*المنافسة|Value)\s*[:\-]?\s*([0-9\.,]+)", text)
    if m: value = _parse_money(m.group(1))
    inq = ""
    m = re.search(r"(?:آخر\s*موعد.*?الاستفسارات|اخر\s*موعد.*?الاستفسارات|Enquir\w*)\s*[:\-]?\s*([0-9/\-]{8,10})", text)
    if m: inq = _parse_date_iso(m.group(1))
    sub = ""
    m = re.search(r"(?:آخر\s*موعد.*?تقديم|اخر\s*موعد.*?التقديم|Submission|Closing)\s*[:\-]?\s*([0-9/\-]{8,10})", text)
    if m: sub = _parse_date_iso(m.group(1))
    link = urljoin(base_url, details_href) if details_href else base_url
    return {
        "اسم المنافسة": name or "",
        "الرقم المرجعي": ref or "",
        "الجهة": entity or "",
        "قيمة المنافسة": value,
        "اخر موعد للاستفسار": inq or "",
        "اخر موعد للتقديم": sub or "",
        "الرابط": link,
    }

def scrape_list_cards(max_pages=6, per_page_delay=0.7, max_rows=250):
    s = _session()
    rows = []
    seen_links = set()
    for base in LIST_PAGES:
        for pn in range(1, max_pages+1):
            try:
                r = _get(s, base, params={"PageNumber": pn}, sleep=per_page_delay)
            except Exception:
                continue
            soup = BeautifulSoup(r.text, "lxml")
            for a in soup.find_all("a", string=lambda x: x and "تفاصيل" in x):
                href = a.get("href") or ""
                if "/Tender/Details" not in href:
                    continue
                card = _closest_card(a)
                rec = _extract_from_card(base, card, href)
                if not rec["اسم المنافسة"] or re.search(r"(بحث|search|المنافسات)$", rec["اسم المنافسة"], re.I):
                    continue
                if rec["الرابط"] in seen_links:
                    continue
                seen_links.add(rec["الرابط"])
                rows.append(rec)
                if len(rows) >= max_rows:
                    break
            if len(rows) >= max_rows:
                break
        if rows:
            break
    df = pd.DataFrame(rows, columns=[
        "اسم المنافسة","الرقم المرجعي","الجهة","قيمة المنافسة",
        "اخر موعد للاستفسار","اخر موعد للتقديم","الرابط"
    ])
    return df

def build_today_df_from_scrape():
    df_all = scrape_list_cards(max_pages=10, max_rows=400)
    if df_all.empty:
        print("List parse produced 0 rows.")
        return df_all.assign(**{"تاريخ_الإدراج": today_ksa_date()})
    df_kw = df_all[df_all.apply(lambda r: _kw_ok(r["اسم المنافسة"], r["الجهة"]), axis=1)].copy()
    out = df_kw if not df_kw.empty else df_all.head(30).copy()
    out["تاريخ_الإدراج"] = today_ksa_date()
    return out

df_today = build_today_df_from_scrape()
if len(df_today) > 0:
    sync_today_sheet(df_today)
    print(f"Rows written today: {len(df_today)}")
else:
    print("No rows extracted.")


In [ ]:
!pip -q install requests beautifulsoup4 lxml dateparser google-api-python-client google-auth


In [ ]:
!pip -q install fuzzywuzzy==0.18.0 python-Levenshtein==0.25.1 arabic_reshaper==3.0.0 python-bidi==0.6.7 reportlab==4.4.4 fpdf2==2.8.4


In [ ]:
!pip freeze > requirements.txt


In [ ]:
# === Open existing Google Sheet by ID (OAuth) ===
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default

SHEET_ID = "1d2js0tZAIUzmVnKwlBHjr3NEvCLfsl6urEhKScXyPME"  # Ghofranai
SHEET_TAB = "Sheet1"  # change if needed

creds, _ = default()
gc = gspread.authorize(creds)
wb = gc.open_by_key(SHEET_ID)
ws = wb.worksheet(SHEET_TAB) if SHEET_TAB in [w.title for w in wb.worksheets()] else wb.sheet1

print("Connected:", wb.title, "→", ws.title)
print("SHEET_ID:", SHEET_ID)
print("Header:", ws.row_values(1))


In [ ]:
import re
import time
import gspread
import google.auth
from google.auth.transport.requests import Request
from google.oauth2.service_account import Credentials

from config import SHEET_ID, SHEET_TAB, SHEET_COLUMNS

SCOPES = [
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive",
]

def get_gspread_client():
    try:
        creds = Credentials.from_service_account_file("credentials.json", scopes=SCOPES)
        return gspread.authorize(creds)
    except Exception:
        creds, _ = google.auth.default(scopes=SCOPES)
        if getattr(creds, "requires_scopes", False):
            creds = creds.with_scopes(SCOPES)
        if getattr(creds, "expired", False) and getattr(creds, "refresh_token", None):
            creds.refresh(Request())
        return gspread.authorize(creds)

gc = get_gspread_client()
sh = gc.open_by_key(SHEET_ID)
ws = sh.worksheet(SHEET_TAB)

try:
    headers = ws.row_values(1)
except gspread.exceptions.APIError:
    headers = []

expected_headers = SHEET_COLUMNS
if headers != expected_headers:
    ws.clear()
    ws.update("A1:G1", [expected_headers])

In [ ]:
# Create a placeholder config.py file
# Replace these with your actual configuration values
config_content = """
SHEET_ID = "YOUR_SHEET_ID"
SHEET_TAB = "YOUR_SHEET_TAB_NAME"
SHEET_COLUMNS = ["اسم المنافسة", "الرقم المرجعي", "الجهة", "قيمة المنافسة", "اخر موعد للاستفسار", "اخر موعد للتقديم", "الرابط", "تاريخ_الإدراج"]
KEYWORDS = ["keyword1", "keyword2", "keyword3"] # Add your keywords here
"""

with open("config.py", "w") as f:
    f.write(config_content)

print("Created placeholder config.py")

In [ ]:
# pip install rapidfuzz  # optional but recommended

import re
import unicodedata

try:
    from rapidfuzz import fuzz as rf_fuzz
    _USE_RAPIDFUZZ = True
except Exception:
    from fuzzywuzzy import fuzz as fw_fuzz
    _USE_RAPIDFUZZ = False

_ARABIC_CHAR_MAP = str.maketrans({
    "إ":"ا","أ":"ا","آ":"ا","ى":"ي","ئ":"ي","ؤ":"و","ة":"ه","ٱ":"ا","ٰ":""  # dagger alef
})

def _strip_diacritics(s: str) -> str:
    return "".join(ch for ch in unicodedata.normalize("NFD", s) if unicodedata.category(ch) != "Mn")

def _norm_ar(s: str) -> str:
    if not isinstance(s, str):
        s = "" if s is None else str(s)
    s = s.strip()
    s = _strip_diacritics(s)
    s = s.replace("ـ", "")                   # tatweel
    s = s.translate(_ARABIC_CHAR_MAP)        # alef/ya/ta marbuta unification
    s = re.sub(r"\s+", " ", s)               # collapse spaces
    s = re.sub(r"[^\w\s]", " ", s)           # drop punctuation
    s = re.sub(r"\s+", " ", s).strip()
    return s.lower()

def _score(a: str, b: str) -> float:
    if _USE_RAPIDFUZZ:
        return max(
            rf_fuzz.ratio(a, b),
            rf_fuzz.partial_ratio(a, b),
            rf_fuzz.token_sort_ratio(a, b),
            rf_fuzz.token_set_ratio(a, b),
        )
    else:
        return max(
            fw_fuzz.ratio(a, b),
            fw_fuzz.partial_ratio(a, b),
            fw_fuzz.token_sort_ratio(a, b),
            fw_fuzz.token_set_ratio(a, b),
        )

def fuzzy_match_keywords(text, keywords, threshold=70):
    """
    Returns (matched: bool, best_keyword: str|None, score: float).
    Arabic-aware normalization + fast substring check before fuzzy.
    """
    if not keywords:
        return False, None, 0.0

    t = _norm_ar(text)
    if not t:
        return False, None, 0.0

    # Pre-normalize keywords once
    norm_kw = [(kw, _norm_ar(kw)) for kw in keywords if isinstance(kw, (str, bytes))]

    # 1) Fast containment check
    for raw_kw, kw in norm_kw:
        if kw and (kw in t or t in kw):
            return True, raw_kw, 100.0

    # 2) Fuzzy best-match
    best_kw, best_sc = None, 0.0
    for raw_kw, kw in norm_kw:
        if not kw:
            continue
        sc = _score(t, kw)
        if sc > best_sc:
            best_sc, best_kw = sc, raw_kw

    if best_sc >= float(threshold):
        return True, best_kw, float(best_sc)
    return False, None, float(best_sc)


In [ ]:
from typing import List, Dict, Any, Tuple
import time

def get_worksheet_data_and_indices_safely(worksheet, expected_headers: List[str] = None) -> Tuple[List[Dict[str, Any]], List[str]]:
    """
    Read all rows from a gspread worksheet with retries.
    Returns:
      - rows: list of dicts (including 'original_row_number')
      - headers: the header row used for mapping
    """
    max_retries = 3
    delay = 1.0

    last_err = None
    for attempt in range(1, max_retries + 1):
        try:
            print(f"Fetching data (attempt {attempt}/{max_retries}) ...")
            all_values = worksheet.get_all_values()

            if not all_values:
                print("Worksheet is empty.")
                return [], []

            headers = all_values[0] if all_values[0] else []
            if expected_headers and headers != expected_headers:
                print("Header mismatch detected. Using sheet headers as-is.")

            if len(all_values) < 2:
                print("No data rows found.")
                return [], headers

            rows: List[Dict[str, Any]] = []
            for row_idx, row in enumerate(all_values[1:], start=2):
                row_dict = {}
                for j, h in enumerate(headers):
                    row_dict[h] = row[j] if j < len(row) else ""
                row_dict["original_row_number"] = row_idx
                rows.append(row_dict)

            print(f"Fetched {len(rows)} rows.")
            return rows, headers

        except Exception as e:
            last_err = e
            print(f"Error reading worksheet: {e}")
            if attempt < max_retries:
                time.sleep(delay)
                delay *= 2

    raise last_err


In [ ]:
from typing import List, Tuple
from fuzzywuzzy import fuzz, process
import os
import time

def fuzzy_match_keywords(text: str, keywords: List[str], threshold: int = 90) -> Tuple[bool, str, int]:
    """
    Return (is_match, best_keyword, score) using token_set_ratio.
    """
    text = (text or "").strip()
    if not text or not keywords:
        return False, "", 0
    best_kw, score = process.extractOne(text, keywords, scorer=fuzz.token_set_ratio)
    return (score >= threshold), best_kw, score

def filter_and_delete_rows(
    worksheet,
    keywords_file: str,
    target_columns: List[str] = None,
    similarity_threshold: int = 90,
    dry_run: bool = True
):
    """
    Filter rows by checking if any of target_columns fuzzy-matches any keyword.
    Rows without a match are deleted (unless dry_run=True).
    """
    if target_columns is None:
        target_columns = ["النشاط الاساسي"]

    # Load keywords
    if not os.path.exists(keywords_file):
        raise FileNotFoundError(f"Keywords file not found: {keywords_file}")
    keywords = read_keywords_from_file(keywords_file)
    if not keywords:
        print("No keywords loaded.")
        return

    # Read sheet rows
    rows, headers = get_worksheet_data_and_indices_safely(worksheet, expected_headers=SHEET_COLUMNS)
    if not rows:
        print("No data rows to process.")
        return

    print(f"Processing {len(rows)} rows")
    print(f"Target columns: {target_columns}")
    print(f"Similarity threshold: {similarity_threshold}%")

    # Validate target columns exist
    missing_cols = [c for c in target_columns if c not in (headers or [])]
    if missing_cols:
        print(f"Warning: target columns missing in sheet: {missing_cols}")

    rows_to_delete = []
    for row in rows:
        keep = False
        for col in target_columns:
            if col in row:
                val = row.get(col, "")
                is_match, best_kw, score = fuzzy_match_keywords(val, keywords, similarity_threshold)
                if is_match:
                    keep = True
                    break
        if not keep:
            rows_to_delete.append(row["original_row_number"])

    print("\n=== SUMMARY ===")
    print(f"Total rows: {len(rows)}")
    print(f"Rows to delete: {len(rows_to_delete)}")
    print(f"Rows to keep: {len(rows) - len(rows_to_delete)}")

    if dry_run:
        print("\n=== DRY RUN ===")
        if rows_to_delete:
            print(f"Rows that would be deleted (desc): {sorted(rows_to_delete, reverse=True)}")
        else:
            print("No rows would be deleted.")
        return

    if not rows_to_delete:
        print("No rows to delete.")
        return

    # Perform deletions (descending order to keep indices stable)
    deleted = 0
    for r in sorted(rows_to_delete, reverse=True):
        try:
            worksheet.delete_rows(r)
            deleted += 1
            time.sleep(0.8)  # rate limit safety
        except Exception as e:
            print(f"Error deleting row {r}: {e}")

    print(f"\nDeleted {deleted} of {len(rows_to_delete)} rows.")


In [ ]:
import os
from config import SHEET_ID as CFG_SHEET_ID, SHEET_TAB as CFG_SHEET_TAB, KEYWORDS as CFG_KEYWORDS

credentials_file = "credentials.json"
sheet_id = "15eOK-kuB2zGOWsNCo1WTu28Xgb8L9Kqzib2UMrHtiwU" or CFG_SHEET_ID
keywords_file = "keywords.txt"

# Ensure keywords file exists (one keyword per line)
if not os.path.exists(keywords_file):
    with open(keywords_file, "w", encoding="utf-8") as f:
        for kw in CFG_KEYWORDS:
            f.write(f"{kw}\n")

print(f"Credentials file: {os.path.abspath(credentials_file)}")
print(f"Sheet ID: {sheet_id}")
print(f"Keywords file: {os.path.abspath(keywords_file)}")

# Quick connectivity check and worksheet handle
client, workbook = setup_connection(credentials_file, sheet_id)
worksheet = open_worksheet(workbook, CFG_SHEET_TAB)


In [ ]:
# Keyword-based row filtering for the new schema (Ghofranai)

TARGET_COLUMNS = ['اسم المنافسة']      # new column to match against
SIMILARITY_THRESHOLD = 95               # keep as needed (90–95 typical)
DRY_RUN = False                         # set True to preview without deleting

# Reuse existing ws if present; otherwise open by SHEET_ID/SHEET_TAB
try:
    _ = ws.title  # ensure ws exists
except NameError:
    from google.colab import auth
    auth.authenticate_user()
    import gspread
    from google.auth import default

    SHEET_ID = "1d2js0tZAIUzmVnKwlBHjr3NEvCLfsl6urEhKScXyPME"  # Ghofranai
    SHEET_TAB = "Sheet1"

    creds, _ = default()
    gc = gspread.authorize(creds)
    wb = gc.open_by_key(SHEET_ID)
    ws = wb.worksheet(SHEET_TAB) if SHEET_TAB in [w.title for w in wb.worksheets()] else wb.sheet1

# Run filtering
try:
    # keywords_file must point to a valid text file with one keyword per line
    filter_and_delete_rows(
        worksheet=ws,
        keywords_file=keywords_file,
        target_columns=TARGET_COLUMNS,
        similarity_threshold=SIMILARITY_THRESHOLD,
        dry_run=DRY_RUN
    )
    print(f"Completed filtering on {TARGET_COLUMNS} | dry_run={DRY_RUN}")
except Exception as e:
    import traceback
    print(f"Filtering failed: {e}")
    traceback.print_exc()


In [ ]:
!pip -q install PyPDF2==3.0.1


In [ ]:
# Robust PDF reader: handles empty pages, whitespace, and encoding quirks
from typing import Optional
import PyPDF2
import os

def read_pdf_content(pdf_path: str) -> Optional[str]:
    """Return concatenated text from all pages in a PDF, or None if failed."""
    if not os.path.exists(pdf_path):
        print(f"Error: File not found: {pdf_path}")
        return None
    try:
        with open(pdf_path, "rb") as fh:
            reader = PyPDF2.PdfReader(fh)
            parts = []
            for i, page in enumerate(reader.pages):
                try:
                    txt = page.extract_text() or ""
                except Exception as e:
                    print(f"Warning: failed to extract page {i}: {e}")
                    txt = ""
                parts.append(txt.strip())
            content = "\n".join([p for p in parts if p])
            content = "\n".join(line.strip() for line in content.splitlines())
            return content if content.strip() else None
    except Exception as e:
        print(f"Error reading PDF: {e}")
        return None

# Example usage
pdf_file_path = "/content/keywords.pdf"
pdf_text_data = read_pdf_content(pdf_file_path)

if pdf_text_data is not None:
    print("Successfully read PDF content into 'pdf_text_data'.")
    # Preview first 500 characters if needed:
    # print(pdf_text_data[:500])
else:
    print("Failed to read PDF content.")


In [ ]:
# Read data directly from Google Sheet (OAuth)
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
import pandas as pd
import numpy as np
import re

SHEET_ID = "1d2js0tZAIUzmVnKwlBHjr3NEvCLfsl6urEhKScXyPME"  # Ghofranai
SHEET_TAB = "Sheet1"  # change if needed

creds, _ = default()
gc = gspread.authorize(creds)
wb = gc.open_by_key(SHEET_ID)
ws = wb.worksheet(SHEET_TAB) if SHEET_TAB in [w.title for w in wb.worksheets()] else wb.sheet1

vals = ws.get_all_values()
if not vals or len(vals) < 1:
    raise ValueError("Sheet appears empty.")

headers = vals[0]
rows = vals[1:]
data = pd.DataFrame(rows, columns=headers)

# Optional: normalize common header variants into the new schema
rename_map = {
    "قيمة الكراسة": "قيمة المنافسة",
    "قيمه وثائق المنافسه": "قيمة المنافسة",
    "قيمة وثائق المنافسة": "قيمة المنافسة",
    "اخر توقيت لاستلام الاستفسارات": "اخر موعد للاستفسار",
    "آخر موعد لاستلام الاستفسارات": "اخر موعد للاستفسار",
    "اخر موعد لتقديم العرض": "اخر موعد للتقديم",
    "آخر موعد لتقديم العرض": "اخر موعد للتقديم",
    "URL": "الرابط",
    "url": "الرابط",
}
present = {k: v for k, v in rename_map.items() if k in data.columns}
if present:
    data = data.rename(columns=present)

# Coerce numeric for "قيمة المنافسة"
if "قيمة المنافسة" in data.columns:
    s = (
        data["قيمة المنافسة"].astype(str)
        .str.replace(r'مجانا|مجان', '0', flags=re.IGNORECASE, regex=True)
        .str.replace(r'[^\d\.\-]', '', regex=True)
        .str.strip()
    )
    data["قيمة المنافسة"] = pd.to_numeric(s, errors="coerce")

# Parse dates if present
for c in ["اخر موعد للاستفسار", "اخر موعد للتقديم", "تاريخ نشرها", "تاريخ النشر"]:
    if c in data.columns:
        data[c] = pd.to_datetime(data[c], errors="coerce")

print(len(data), "rows")
display(data.head(10))


In [ ]:
# --- 0. EMAIL MODULES ---
import smtplib
import ssl
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders

# --- 0. REPORTLAB & ANALYSIS MODULES (Standard) ---
from reportlab.lib.pagesizes import A4
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle, PageBreak, Image as RlImage
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch
from reportlab.lib import colors
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
import arabic_reshaper
from bidi.algorithm import get_display

# Data Analysis Modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import warnings
import re
from matplotlib.font_manager import FontProperties
import io
import unicodedata

warnings.filterwarnings('ignore')

# --- 1. GLOBAL CONFIGURATIONS (Shared) ---
FONT_FILENAME = 'Amiri-Regular.ttf'
ARABIC_FONT_PATH = FONT_FILENAME
PLOT_DPI = 150

# --- EMAIL CONFIGURATION ---
SMTP_SERVER = "smtp.gmail.com"
SMTP_PORT = 465
SENDER_EMAIL = "ghofranzalqarni@gmail.com"
SENDER_PASSWORD = "euzysjzccifszhsr"  # Gmail App Password recommended

# Register Arabic font for ReportLab
try:
    pdfmetrics.registerFont(TTFont('ArabicFont', FONT_FILENAME))
except Exception as e:
    print(f"Error registering ReportLab font: {e}. Check if '{FONT_FILENAME}' exists in the working directory.")

# --- 2. ARABIC TEXT UTILITIES (Shared) ---
def _prepare_arabic_text(text):
    if not isinstance(text, str):
        return text
    reshaped_text = arabic_reshaper.reshape(text)
    bidi_text = get_display(reshaped_text)
    return bidi_text

# Minimal Arabic normalization for grouping/keys
_ARABIC_MAP = str.maketrans({
    "أ": "ا", "إ": "ا", "آ": "ا",
    "ى": "ي", "ي": "ي",
    "ة": "ه",
    "ؤ": "و", "ئ": "ي",
})
_ARABIC_DIACRITICS = {chr(c) for c in range(0x0610, 0x061B+1)} | {chr(c) for c in range(0x064B, 0x065F+1)}

def _strip_diacritics(s: str) -> str:
    return "".join(ch for ch in s if ch not in _ARABIC_DIACRITICS)

def _normalize_arabic(s):
    if not isinstance(s, str):
        return s
    s = unicodedata.normalize("NFKC", s)
    s = _strip_diacritics(s)
    s = s.translate(_ARABIC_MAP)
    s = re.sub(r"\s+", " ", s).strip()
    return s

# --- 3. REPORTLAB STYLES ---
styles = getSampleStyleSheet()
arabic_style_title = ParagraphStyle(
    name='ArabicTitle', parent=styles['Title'], fontName='ArabicFont', fontSize=18, alignment=2, spaceAfter=24, textColor=colors.darkblue
)
arabic_style_heading2 = ParagraphStyle(
    name='ArabicHeading2', parent=styles['Heading2'], fontName='ArabicFont', fontSize=14, alignment=2, spaceBefore=18, spaceAfter=12, textColor=colors.black
)
arabic_style_normal = ParagraphStyle(
    name='ArabicNormal', parent=styles['Normal'], fontName='ArabicFont', fontSize=12, alignment=2, spaceAfter=6, textColor=colors.black
)

# --- 4. DATA ANALYSIS CLASS ---
ARABIC_FONT_PROP = None

def _get_arabic_font_prop():
    global ARABIC_FONT_PROP
    if ARABIC_FONT_PROP is None:
        try:
            ARABIC_FONT_PROP = FontProperties(fname=ARABIC_FONT_PATH)
        except FileNotFoundError:
            ARABIC_FONT_PROP = FontProperties(family='sans-serif')
    return ARABIC_FONT_PROP

class TenderDataAnalyzer:
    def __init__(self):
        self.df = None
        self.font_prop = _get_arabic_font_prop()
        self.plot_paths = {}

    def load_data_from_text(self, data_text):
        self.df = pd.DataFrame(data_text)

        date_columns = ['اخر معاد لاستلام الاستفسارات', 'اخر موعد لتقديم العرض', 'تاريخ النشر']
        for col in date_columns:
            if col in self.df.columns:
                self.df[col] = pd.to_datetime(self.df[col], errors='coerce')

        value_col = 'قيمه وثائق المنافسه'
        if value_col in self.df.columns:
            self.df[value_col] = (
                self.df[value_col]
                .astype(str)
                .str.replace(r'مجانا|مجان', '0', flags=re.IGNORECASE, regex=True)
                .str.replace(r'[^\d\.\-]', '', regex=True)
                .str.strip()
            )
            self.df[value_col] = pd.to_numeric(self.df[value_col], errors='coerce')

        for col in ['النشاط الاساسي', 'النشاط الأساسي', 'العنوان']:
            if col in self.df.columns:
                new_col = f'{col}_normalized'
                self.df[new_col] = self.df[col].apply(_normalize_arabic)

        return self.df

    def group_by_activity(self):
        activity_col = None
        possible_names = ['النشاط الاساسي_normalized', 'النشاط الأساسي_normalized', 'النشاط الاساسي', 'النشاط الأساسي']
        for col_name in possible_names:
            if col_name in self.df.columns:
                activity_col = col_name
                break
        if activity_col is None:
            return None, None

        activity_counts = self.df[activity_col].value_counts()
        activity_stats = None
        value_col = 'قيمه وثائق المنافسه'

        if value_col in self.df.columns:
            activity_stats = self.df.groupby(activity_col).agg(
                Count=(value_col, 'count'),
                MeanValue=(value_col, 'mean'),
                TotalValue=(value_col, 'sum'),
                MinValue=(value_col, 'min'),
                MaxValue=(value_col, 'max')
            ).round(2)

        plt.figure(figsize=(12, 6))

        plt.subplot(1, 2, 1)
        labels = [_prepare_arabic_text(label) for label in activity_counts.index]
        title = _prepare_arabic_text('عدد المنافسات حسب النشاط الأساسي')
        ylabel = _prepare_arabic_text('عدد المنافسات')
        plt.bar(range(len(activity_counts)), activity_counts.values)
        plt.title(title, fontproperties=self.font_prop)
        plt.xticks(range(len(activity_counts)), labels, rotation=45, ha='right', fontproperties=self.font_prop)
        plt.ylabel(ylabel, fontproperties=self.font_prop)

        if value_col in self.df.columns:
            plt.subplot(1, 2, 2)
            activity_values = self.df.groupby(activity_col)[value_col].sum()
            pie_labels = [_prepare_arabic_text(label) for label in activity_values.index]
            pie_title = _prepare_arabic_text('توزيع قيمة الكراسه')
            plt.pie(activity_values, labels=pie_labels, autopct='%1.1f%%',
                    textprops={'fontproperties': self.font_prop, 'fontsize': 10})
            plt.title(pie_title, fontproperties=self.font_prop)
            plt.ylabel('')
            plt.tight_layout()

        buf = io.BytesIO()
        plt.savefig(buf, format='png', dpi=PLOT_DPI)
        plt.close()
        buf.seek(0)
        self.plot_paths['activity_plot'] = buf

        return activity_counts, activity_stats

    def group_by_organization(self):
        org_col = 'العنوان_normalized' if 'العنوان_normalized' in self.df.columns else 'العنوان'
        if org_col not in self.df.columns:
            return None

        org_counts = self.df[org_col].value_counts()
        top_orgs = org_counts.head(10)

        plt.figure(figsize=(12, 8))
        labels = [_prepare_arabic_text(org) for org in top_orgs.index]
        labels = [org[:50] + '...' if isinstance(org, str) and len(org) > 50 else org for org in labels]
        xlabel = _prepare_arabic_text('عدد المنافسات')
        title = _prepare_arabic_text('أكثر 10 جهات إصداراً للمناقصات')

        plt.barh(range(len(top_orgs)), top_orgs.values)
        plt.yticks(range(len(top_orgs)), labels, fontproperties=self.font_prop)
        plt.xlabel(xlabel, fontproperties=self.font_prop)
        plt.title(title, fontproperties=self.font_prop)
        plt.gca().invert_yaxis()
        plt.tight_layout()

        buf = io.BytesIO()
        plt.savefig(buf, format='png', dpi=PLOT_DPI)
        plt.close()
        buf.seek(0)
        self.plot_paths['organization_plot'] = buf

        return org_counts

    def analyze_by_date(self):
        date_col = 'تاريخ النشر'
        if date_col not in self.df.columns:
            return None

        self.df['شهر النشر'] = self.df[date_col].dt.to_period('M')
        monthly_counts = self.df['شهر النشر'].value_counts().sort_index()

        plt.figure(figsize=(14, 6))

        plt.subplot(1, 2, 1)
        title1 = _prepare_arabic_text('اتجاه نشر المنافسات شهرياً')
        xlabel1 = _prepare_arabic_text('الشهر')
        ylabel1 = _prepare_arabic_text('عدد المنافسات')
        monthly_counts.plot(kind='line', marker='o')
        plt.title(title1, fontproperties=self.font_prop)
        plt.xlabel(xlabel1, fontproperties=self.font_prop)
        plt.ylabel(ylabel1, fontproperties=self.font_prop)
        plt.xticks(rotation=45)

        plt.subplot(1, 2, 2)
        self.df['يوم الأسبوع'] = self.df[date_col].dt.dayofweek
        day_names_ar = ['الاثنين', 'الثلاثاء', 'الأربعاء', 'الخميس', 'الجمعة', 'السبت', 'الأحد']
        day_counts = self.df['يوم الأسبوع'].value_counts().sort_index()

        title2 = _prepare_arabic_text('توزيع النشر حسب أيام الأسبوع')
        ylabel2 = _prepare_arabic_text('عدد المنافسات')
        xlabels = [_prepare_arabic_text(d) for d in [day_names_ar[i] for i in day_counts.index]]
        plt.bar(xlabels, day_counts.values)
        plt.title(title2, fontproperties=self.font_prop)
        plt.xticks(rotation=45, fontproperties=self.font_prop)
        plt.ylabel(ylabel2, fontproperties=self.font_prop)

        plt.tight_layout()

        buf = io.BytesIO()
        plt.savefig(buf, format='png', dpi=PLOT_DPI)
        plt.close()
        buf.seek(0)
        self.plot_paths['date_analysis_plot'] = buf

        return monthly_counts

    def analyze_weekly_tender_count_trend(self):
        date_col = 'تاريخ النشر'
        if date_col not in self.df.columns:
            return None

        self.df['أسبوع النشر'] = self.df[date_col].dt.to_period('W')
        weekly_count = self.df['أسبوع النشر'].value_counts().sort_index()

        plt.figure(figsize=(12, 6))
        title = _prepare_arabic_text('الاتجاه الأسبوعي لعدد المنافسات')
        xlabel = _prepare_arabic_text('أسبوع النشر')
        ylabel = _prepare_arabic_text('عدد المنافسات')
        weekly_count.plot(kind='line', marker='o', linestyle='-')
        plt.title(title, fontproperties=self.font_prop)
        plt.xlabel(xlabel, fontproperties=self.font_prop)
        plt.ylabel(ylabel, fontproperties=self.font_prop)
        plt.grid(True, linestyle='--', alpha=0.6)
        plt.xticks(rotation=45)
        plt.tight_layout()

        buf = io.BytesIO()
        plt.savefig(buf, format='png', dpi=PLOT_DPI)
        plt.close()
        buf.seek(0)
        self.plot_paths['weekly_count_trend_plot'] = buf

        return weekly_count

    def analyze_weekly_document_value_trend(self):
        date_col = 'تاريخ النشر'
        value_col = 'قيمه وثائق المنافسه'
        if date_col not in self.df.columns or value_col not in self.df.columns:
            return None

        self.df['أسبوع النشر'] = self.df[date_col].dt.to_period('W')
        weekly_value_sum = self.df.groupby('أسبوع النشر')[value_col].sum().sort_index()

        plt.figure(figsize=(12, 6))
        title = _prepare_arabic_text('الاتجاه الأسبوعي لإجمالي قيمة الكراسه')
        xlabel = _prepare_arabic_text('أسبوع النشر')
        ylabel = _prepare_arabic_text('إجمالي قيمة الكراسه')
        weekly_value_sum.plot(kind='line', marker='o', linestyle='-')
        plt.title(title, fontproperties=self.font_prop)
        plt.xlabel(xlabel, fontproperties=self.font_prop)
        plt.ylabel(ylabel, fontproperties=self.font_prop)
        plt.grid(True, linestyle='--', alpha=0.6)
        plt.xticks(rotation=45)
        plt.tight_layout()

        buf = io.BytesIO()
        plt.savefig(buf, format='png', dpi=PLOT_DPI)
        plt.close()
        buf.seek(0)
        self.plot_paths['weekly_value_trend_plot'] = buf

        return weekly_value_sum

    def analyze_document_values(self):
        value_col = 'قيمه وثائق المنافسه'
        if value_col not in self.df.columns:
            return None, None

        values = self.df[value_col].dropna()
        summary_stats = {
            'Total Value': values.sum(),
            'Mean Value': values.mean(),
            'Max Value': values.max(),
            'Min Value': values.min()
        }

        bins = [0, 500, 1000, 2000, 5000, float('inf')]
        labels = ['0-500', '501-1000', '1001-2000', '2001-5000', '5000+']
        self.df['فئة القيمة'] = pd.cut(self.df[value_col], bins=bins, labels=labels, include_lowest=True)
        value_distribution = self.df['فئة القيمة'].value_counts().sort_index()

        plt.figure(figsize=(14, 6))

        plt.subplot(1, 2, 1)
        title1 = _prepare_arabic_text('توزيع قيم الكراسه المنافسات')
        xlabel1 = _prepare_arabic_text('قيمة الكراسه')
        ylabel1 = _prepare_arabic_text('التكرار')
        plt.hist(values, bins=20, edgecolor='black', alpha=0.7)
        plt.title(title1, fontproperties=self.font_prop)
        plt.xlabel(xlabel1, fontproperties=self.font_prop)
        plt.ylabel(ylabel1, fontproperties=self.font_prop)

        plt.subplot(1, 2, 2)
        pie_labels = [_prepare_arabic_text(str(label)) for label in value_distribution.index]
        pie_title = _prepare_arabic_text('توزيع المنافسات حسب فئات القيمة')
        plt.pie(value_distribution, labels=pie_labels, autopct='%1.1f%%',
                textprops={'fontproperties': self.font_prop, 'fontsize': 10})
        plt.title(pie_title, fontproperties=self.font_prop)
        plt.ylabel('')

        plt.tight_layout()

        buf = io.BytesIO()
        plt.savefig(buf, format='png', dpi=PLOT_DPI)
        plt.close()
        buf.seek(0)
        self.plot_paths['value_analysis_plot'] = buf

        return summary_stats, value_distribution

    def generate_summary_report(self):
        summary = {}
        summary['Total Tenders'] = len(self.df)

        activity_col = None
        possible_activity_names = ['النشاط الاساسي_normalized', 'النشاط الأساسي_normalized', 'النشاط الاساسي', 'النشاط الأساسي']
        for col_name in possible_activity_names:
            if col_name in self.df.columns:
                activity_col = col_name
                break

        org_col = 'العنوان_normalized' if 'العنوان_normalized' in self.df.columns else 'العنوان'

        if activity_col:
            summary['Unique Activities'] = self.df[activity_col].nunique()
            mode_value = self.df[activity_col].mode()
            summary['Most Common Activity'] = _prepare_arabic_text(str(mode_value.iloc[0])) if not mode_value.empty else 'N/A'

        if org_col in self.df.columns:
            summary['Unique Organizations'] = self.df[org_col].nunique()

        if 'قيمه وثائق المنافسه' in self.df.columns:
            valid_values = self.df['قيمه وثائق المنافسه'].dropna()
            summary['Total Value'] = valid_values.sum()
            summary['Average Value'] = valid_values.mean()
            summary['Min Value'] = valid_values.min()
            summary['Max Value'] = valid_values.max()

        if 'تاريخ النشر' in self.df.columns and not self.df['تاريخ النشر'].isnull().all():
            date_range = self.df['تاريخ النشر'].max() - self.df['تاريخ النشر'].min()
            summary['Time Period Days'] = date_range.days

        return summary

# --- 5. REPORT GENERATION FUNCTION ---
def df_to_table_data(df, include_index=False):
    header = [_prepare_arabic_text(str(col)) for col in df.columns]
    if include_index:
        index_name = str(df.index.name) if df.index.name else ''
        header.insert(0, _prepare_arabic_text(index_name))

    data = [header]
    for index, row in df.iterrows():
        row_list = []
        if include_index:
            row_list.append(_prepare_arabic_text(str(index)))
        for item in row.values:
            if isinstance(item, (int, float, np.number)):
                if isinstance(item, int):
                    row_list.append(f"{item:,}")
                else:
                    if item == int(item):
                        row_list.append(f"{int(item):,}")
                    else:
                        row_list.append(f"{item:,.2f}")
            else:
                row_list.append(_prepare_arabic_text(str(item)))
        data.append(row_list)
    return data

def create_full_pdf_report(analyzer: TenderDataAnalyzer, filename='arabic_data_report.pdf'):
    doc = SimpleDocTemplate(
        filename, pagesize=A4, rightMargin=72, leftMargin=72, topMargin=72, bottomMargin=18
    )
    story = []
    table_style = TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor('#ADD8E6')),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.black),
        ('ALIGN', (0, 0), (-1, -1), 'RIGHT'),
        ('FONTNAME', (0, 0), (-1, -1), 'ArabicFont'),
        ('FONTSIZE', (0, 0), (-1, -1), 10),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 6),
        ('GRID', (0, 0), (-1, -1), 1, colors.black),
        ('BACKGROUND', (0, 1), (-1, -1), colors.white),
    ])

    story.append(Paragraph(_prepare_arabic_text('تقرير تحليلي شامل للمناقصات'), arabic_style_title))
    story.append(Spacer(1, 0.5 * inch))
    story.append(Paragraph(_prepare_arabic_text(f'تاريخ التقرير: {datetime.now().strftime("%Y-%m-%d")}'), arabic_style_normal))
    story.append(PageBreak())

    story.append(Paragraph(_prepare_arabic_text('1. الملخص التنفيذي'), arabic_style_heading2))
    summary = analyzer.generate_summary_report()

    summary_data = [
        [_prepare_arabic_text('الإجمالي'), _prepare_arabic_text('القيمة')],
        [_prepare_arabic_text('إجمالي عدد المنافسات'), f"{summary.get('Total Tenders', 'N/A'):,}" if isinstance(summary.get('Total Tenders'), (int, float)) else str(summary.get('Total Tenders', 'N/A'))],
        [_prepare_arabic_text('عدد الأنشطة المختلفة'), f"{summary.get('Unique Activities', 'N/A'):,}" if isinstance(summary.get('Unique Activities'), (int, float)) else str(summary.get('Unique Activities', 'N/A'))],
        [_prepare_arabic_text('النشاط الأكثر شيوعاً'), summary.get('Most Common Activity', 'N/A')],
        [_prepare_arabic_text('عدد الجهات'), f"{summary.get('Unique Organizations', 'N/A'):,}" if isinstance(summary.get('Unique Organizations'), (int, float)) else str(summary.get('Unique Organizations', 'N/A'))],
        [_prepare_arabic_text('إجمالي قيمة الكراسه'), f"{summary.get('Total Value', 'N/A'):,.0f}" if isinstance(summary.get('Total Value'), (int, float)) else str(summary.get('Total Value', 'N/A'))],
        [_prepare_arabic_text('متوسط قيمة الكراسه'), f"{summary.get('Average Value', 'N/A'):,.0f}" if isinstance(summary.get('Average Value'), (int, float)) else str(summary.get('Average Value', 'N/A'))],
        [_prepare_arabic_text('أعلى قيمة كراسه'), f"{summary.get('Max Value', 'N/A'):,.0f}" if isinstance(summary.get('Max Value'), (int, float)) else str(summary.get('Max Value', 'N/A'))],
        [_prepare_arabic_text('أقل قيمة كراسه'), f"{summary.get('Min Value', 'N/A'):,.0f}" if isinstance(summary.get('Min Value'), (int, float)) else str(summary.get('Min Value', 'N/A'))],
        [_prepare_arabic_text('الفترة الزمنية (يوم)'), f"{summary.get('Time Period Days', 'N/A'):,}" if isinstance(summary.get('Time Period Days'), (int, float)) else str(summary.get('Time Period Days', 'N/A'))]
    ]
    summary_table = Table(summary_data, colWidths=[2.5 * inch, 2.5 * inch])
    summary_table.setStyle(table_style)
    story.append(Paragraph(_prepare_arabic_text('ملخص البيانات:'), arabic_style_normal))
    story.append(summary_table)
    story.append(Spacer(1, 0.25 * inch))

    story.append(Paragraph(_prepare_arabic_text('2. التحليل حسب النشاط الأساسي'), arabic_style_heading2))
    _, activity_stats = analyzer.group_by_activity()

    if activity_stats is not None and not activity_stats.empty:
        story.append(Paragraph(_prepare_arabic_text('إحصائيات مفصلة حسب النشاط:'), arabic_style_normal))
        activity_table_data = df_to_table_data(activity_stats, include_index=True)
        num_activity_cols = len(activity_stats.columns) + 1
        activity_table = Table(activity_table_data, colWidths=[2 * inch] + [5.5 * inch / (num_activity_cols - 1)] * (num_activity_cols - 1))
        activity_table.setStyle(table_style)
        story.append(activity_table)
        story.append(Spacer(1, 0.25 * inch))
    elif activity_stats is None:
        story.append(Paragraph(_prepare_arabic_text('بيانات النشاط الأساسي غير متوفرة.'), arabic_style_normal))
    else:
        story.append(Paragraph(_prepare_arabic_text('لا توجد إحصائيات نشاط لعرضها.'), arabic_style_normal))

    if 'activity_plot' in analyzer.plot_paths and analyzer.plot_paths['activity_plot'].getbuffer().nbytes > 0:
        story.append(Paragraph(_prepare_arabic_text('مقارنة عدد المناقصات وقيمة الكراسات حسب النشاط:'), arabic_style_normal))
        img = RlImage(analyzer.plot_paths['activity_plot'], width=6*inch, height=3*inch)
        story.append(img)
    else:
        story.append(Paragraph(_prepare_arabic_text('الرسم البياني للنشاط غير متوفر.'), arabic_style_normal))

    story.append(PageBreak())

    story.append(Paragraph(_prepare_arabic_text('3. التحليل حسب الجهة المصدرة'), arabic_style_heading2))
    org_counts = analyzer.group_by_organization()

    if org_counts is not None and not org_counts.empty:
        top_10_orgs = org_counts.head(10).to_frame(name='Count')
        story.append(Paragraph(_prepare_arabic_text('أكثر 10 جهات إصداراً للمناقصات:'), arabic_style_normal))
        org_table_data = df_to_table_data(top_10_orgs, include_index=True)
        org_table = Table(org_table_data, colWidths=[4 * inch, 1.5 * inch])
        org_table.setStyle(table_style)
        story.append(org_table)
        story.append(Spacer(1, 0.25 * inch))
    elif org_counts is None:
        story.append(Paragraph(_prepare_arabic_text('بيانات الجهة المصدرة غير متوفرة.'), arabic_style_normal))
    else:
        story.append(Paragraph(_prepare_arabic_text('لا توجد بيانات جهات لعرضها.'), arabic_style_normal))

    if 'organization_plot' in analyzer.plot_paths and analyzer.plot_paths['organization_plot'].getbuffer().nbytes > 0:
        story.append(Paragraph(_prepare_arabic_text('توزيع المناقصات حسب الجهة (أعلى 10):'), arabic_style_normal))
        img = RlImage(analyzer.plot_paths['organization_plot'], width=6*inch, height=4*inch)
        story.append(img)
    else:
        story.append(Paragraph(_prepare_arabic_text('الرسم البياني للجهات غير متوفر.'), arabic_style_normal))

    story.append(PageBreak())

    story.append(Paragraph(_prepare_arabic_text('4. التحليل الزمني'), arabic_style_heading2))
    analyzer.analyze_by_date()

    if 'date_analysis_plot' in analyzer.plot_paths and analyzer.plot_paths['date_analysis_plot'].getbuffer().nbytes > 0:
        story.append(Paragraph(_prepare_arabic_text('اتجاهات النشر الشهري وتوزيعها حسب أيام الأسبوع:'), arabic_style_normal))
        img = RlImage(analyzer.plot_paths['date_analysis_plot'], width=6.5*inch, height=3*inch)
        story.append(img)
    else:
        story.append(Paragraph(_prepare_arabic_text('الرسم البياني للتحليل الزمني غير متوفر.'), arabic_style_normal))

    story.append(Spacer(1, 0.5 * inch))

    analyzer.analyze_weekly_tender_count_trend()
    if 'weekly_count_trend_plot' in analyzer.plot_paths and analyzer.plot_paths['weekly_count_trend_plot'].getbuffer().nbytes > 0:
        story.append(Paragraph(_prepare_arabic_text('الاتجاه الأسبوعي لعدد المنافسات:'), arabic_style_normal))
        img = RlImage(analyzer.plot_paths['weekly_count_trend_plot'], width=6.5*inch, height=3.5*inch)
        story.append(img)
    else:
        story.append(Paragraph(_prepare_arabic_text('الرسم البياني لاتجاه العدد الأسبوعي غير متوفر.'), arabic_style_normal))

    story.append(PageBreak())

    story.append(Paragraph(_prepare_arabic_text('5. تحليل قيم الكراسه'), arabic_style_heading2))
    value_stats, value_dist = analyzer.analyze_document_values()

    if value_stats and any(isinstance(v, (int, float)) for v in value_stats.values()):
        value_data_rows = [
            [_prepare_arabic_text('إجمالي قيمة الكراسه'), f"{value_stats.get('Total Value', 'N/A'):,.0f}" if isinstance(value_stats.get('Total Value'), (int, float)) else str(value_stats.get('Total Value', 'N/A'))],
            [_prepare_arabic_text('متوسط قيمة الكراسه'), f"{value_stats.get('Average Value', 'N/A'):,.0f}" if isinstance(value_stats.get('Average Value'), (int, float)) else str(value_stats.get('Average Value', 'N/A'))],
            [_prepare_arabic_text('أعلى قيمة'), f"{value_stats.get('Max Value', 'N/A'):,.0f}" if isinstance(value_stats.get('Max Value'), (int, float)) else str(value_stats.get('Max Value', 'N/A'))],
            [_prepare_arabic_text('أقل قيمة'), f"{value_stats.get('Min Value', 'N/A'):,.0f}" if isinstance(value_stats.get('Min Value'), (int, float)) else str(value_stats.get('Min Value', 'N/A'))]
        ]
        value_table = Table(value_data_rows, colWidths=[2.5 * inch, 2.5 * inch])
        value_table.setStyle(table_style)
        story.append(Paragraph(_prepare_arabic_text('ملخص إحصائيات قيم الكراسه:'), arabic_style_normal))
        story.append(value_table)
        story.append(Spacer(1, 0.25 * inch))
    else:
        story.append(Paragraph(_prepare_arabic_text('إحصائيات قيم الكراسه غير متوفرة.'), arabic_style_normal))

    if 'value_analysis_plot' in analyzer.plot_paths and analyzer.plot_paths['value_analysis_plot'].getbuffer().nbytes > 0:
        story.append(Paragraph(_prepare_arabic_text('توزيع القيم الإجمالية حسب الفئات:'), arabic_style_normal))
        img = RlImage(analyzer.plot_paths['value_analysis_plot'], width=6.5*inch, height=3*inch)
        story.append(img)
    else:
        story.append(Paragraph(_prepare_arabic_text('الرسم البياني لتوزيع قيم الكراسه غير متوفر.'), arabic_style_normal))

    story.append(Spacer(1, 0.5 * inch))
    story.append(Paragraph(_prepare_arabic_text('Hued & Araamis report'), arabic_style_normal))

    doc.build(story)
    print(f"Report generated: {filename}")
    return filename

# --- 6. EMAIL SENDING FUNCTION ---
def send_email_with_pdf(pdf_filepath, recipient_email):
    if not SENDER_EMAIL or not SENDER_PASSWORD:
        print("Email settings are missing.")
        return

    message = MIMEMultipart()
    message["From"] = SENDER_EMAIL
    message["To"] = recipient_email
    message["Subject"] = "تقرير تحليل المنافسات"

    body = "مرفق تقرير تحليل المنافسات بصيغة PDF."
    message.attach(MIMEText(body, "plain", "utf-8"))

    try:
        with open(pdf_filepath, "rb") as attachment:
            part = MIMEBase("application", "octet-stream")
            part.set_payload(attachment.read())
        encoders.encode_base64(part)
        part.add_header("Content-Disposition", f"attachment; filename={pdf_filepath}")
        message.attach(part)
    except FileNotFoundError:
        print(f"Attachment not found: {pdf_filepath}")
        return
    except Exception as e:
        print(f"Error attaching file: {e}")
        return

    try:
        context = ssl.create_default_context()
        with smtplib.SMTP_SSL(SMTP_SERVER, SMTP_PORT, context=context) as server:
            server.login(SENDER_EMAIL, SENDER_PASSWORD)
            server.sendmail(SENDER_EMAIL, recipient_email, message.as_string())
        print(f"Email sent to {recipient_email}")
    except Exception as e:
        print(f"SMTP error: {e}")


In [ ]:
# Build analyzer and load data (expects a DataFrame-compatible object)
analyzer = TenderDataAnalyzer()
analyzer.load_data_from_text(data)

# Generate PDF report
REPORT_FILE = f"Tender_Report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.pdf"
generated_pdf_path = create_full_pdf_report(analyzer, REPORT_FILE)

# Send report by email
RECIPIENT_EMAIL = "basma66yy@gmail.com"
send_email_with_pdf(generated_pdf_path, RECIPIENT_EMAIL)

print(generated_pdf_path)

##Machine learning

In [ ]:
for i, col_name in enumerate(list(data.columns)):
    print(f"Index: {i}, Column Name: {col_name}")


In [ ]:
import pandas as pd
import numpy as np
import re
from typing import Tuple

def preprocess_data(df_main: pd.DataFrame, df_target: pd.DataFrame) -> Tuple[pd.DataFrame, pd.Series]:
    """
    Merge, clean, and engineer features for tender data.

    Requirements/assumptions:
      - df_main contains tender fields including:
        ['الرقم المرجعي','النشاط الاساسي','العنوان','وصف',
         'قيمه وثائق المنافسه','اخر موعد لتقديم العرض','تاريخ النشر']
        (some may be missing; code guards accordingly)
      - df_target contains ['الرقم المرجعي','حالة القبول'] (target label)

    Returns:
      X (features DataFrame), y (target Series)
    """
    print("=" * 60)
    print("1) Merge and basic cleanup")

    # Defensive copies
    df_main = df_main.copy()
    df_target = df_target.copy()

    # Ensure required merge keys exist
    if 'الرقم المرجعي' not in df_main.columns or 'الرقم المرجعي' not in df_target.columns:
        raise KeyError("Both dataframes must include 'الرقم المرجعي' for merging.")

    if 'حالة القبول' not in df_target.columns:
        raise KeyError("df_target must include 'حالة القبول' as the classification label.")

    # Inner join
    df = pd.merge(df_main, df_target[['الرقم المرجعي', 'حالة القبول']], on='الرقم المرجعي', how='inner')
    print(f"Rows after inner join: {len(df)}")

    # Normalize common alternative column names
    # Map alternatives to canonical names used below
    rename_map = {
        'اخر معاد لاستلام الاستفسارات': 'اخر موعد لاستلام الاستفسارات',
        'قيمة وثائق المنافسة': 'قيمه وثائق المنافسه',
        'قيمة وثائق المنافسه': 'قيمه وثائق المنافسه',
        'URL': 'url',
        'الرابط': 'url'
    }
    present_map = {k: v for k, v in rename_map.items() if k in df.columns}
    if present_map:
        df = df.rename(columns=present_map)

    # Drop unneeded columns if they exist
    to_drop = [c for c in ['url', 'اخر موعد لاستلام الاستفسارات'] if c in df.columns]
    if to_drop:
        df = df.drop(columns=to_drop)
        print(f"Dropped columns: {to_drop}")
    print("-" * 40)

    print("2) Value cleaning and date features")

    # Clean numeric field: 'قيمه وثائق المنافسه'
    val_col = 'قيمه وثائق المنافسه'
    if val_col in df.columns:
        # Replace Arabic "free" variants with 0
        ser = (
            df[val_col]
            .astype(str)
            .str.replace(r'مجانا|مجان', '0', flags=re.IGNORECASE, regex=True)
            # Strip any non-digit separators except dot and minus
            .str.replace(r'[^\d\.\-]', '', regex=True)
            .str.strip()
        )
        df[val_col] = pd.to_numeric(ser, errors='coerce').fillna(0)
    else:
        df[val_col] = 0.0

    # Parse dates (coerce invalids)
    date_cols = ['اخر موعد لتقديم العرض', 'تاريخ النشر']
    for c in date_cols:
        if c in df.columns:
            df[c] = pd.to_datetime(df[c], errors='coerce')
        else:
            df[c] = pd.NaT

    # Feature: days remaining from publish to deadline (can be negative if data inconsistent)
    df['ايام_متبقية_للعرض'] = (df['اخر موعد لتقديم العرض'] - df['تاريخ النشر']).dt.days
    df['ايام_متبقية_للعرض'] = df['ايام_متبقية_للعرض'].fillna(0).astype(int)

    # Feature: free flag
    df['مجانية_المنافسة'] = (df[val_col] == 0).astype(int)

    # Remove raw dates if not needed further
    df = df.drop(columns=[c for c in date_cols if c in df.columns], errors='ignore')
    print("Engineered: 'ايام├متبقية├للعرض', 'مجانية├المنافسة' and cleaned value column.")
    print("-" * 40)

    print("3) Categorical encoding")

    # Ensure categorical columns exist; create if absent to keep pipeline stable
    for c in ['النشاط الاساسي', 'العنوان', 'وصف']:
        if c not in df.columns:
            df[c] = ""

    # Fill NaNs then one-hot encode (drop_first to limit dummy explosion)
    cat_cols = ['النشاط الاساسي', 'العنوان', 'وصف']
    df[cat_cols] = df[cat_cols].fillna("").astype(str)
    df = pd.get_dummies(df, columns=cat_cols, drop_first=True)

    # Target encoding
    # Map to binary (customize if different labels are used)
    label_map = {'مقبول': 1, 'مرفوض': 0}
    y = df['حالة القبول'].map(label_map)
    if y.isna().any():
        # Fallback: treat any non-مرفوض as 1, only explicit مرفوض as 0
        y = (df['حالة القبول'] != 'مرفوض').astype(int)
    df = df.drop(columns=['حالة القبول'])

    # Remove identifier columns not intended as features
    if 'الرقم المرجعي' in df.columns:
        df = df.drop(columns=['الرقم المرجعي'])

    X = df

    print(f"Final shape -> X: {X.shape}, y: {y.shape}")
    print(f"Feature sample: {list(X.columns)[:8]} ...")
    print("=" * 60)
    return X, y


In [ ]:
import pandas as pd

# Arabic rendering helper (safe fallback if libraries غير متاحة)
def _prepare_arabic_text(text):
    try:
        import arabic_reshaper
        from bidi.algorithm import get_display
        if not isinstance(text, str):
            return text
        if not text.strip():
            return text
        return get_display(arabic_reshaper.reshape(text))
    except Exception:
        return text

def get_column_total_values(df: pd.DataFrame) -> dict:
    """
    يجمع قيمة الأعمدة الرقمية. لو العمود نصي لكنه أرقام (مثلاً 'مجانا' أو بها رموز)،
    يحاول تحويله لأرقام ثم يجمعه.
    """
    totals = {}
    for col in df.columns:
        s = df[col]
        if pd.api.types.is_numeric_dtype(s):
            totals[col] = float(pd.to_numeric(s, errors='coerce').fillna(0).sum())
        else:
            # محاولة تحويل النص لأرقام (إزالة غير الأرقام و'مجانا' -> 0)
            try:
                ser = (
                    s.astype(str)
                     .str.replace(r'مجانا|مجان', '0', regex=True)
                     .str.replace(r'[^\d\.\-]', '', regex=True)
                     .str.strip()
                )
                nums = pd.to_numeric(ser, errors='coerce').fillna(0)
                if nums.notna().any() and nums.sum() != 0:
                    totals[col] = float(nums.sum())
            except Exception:
                pass
    return totals


In [ ]:
# Compute totals and pretty-print (sorted descending)
column_name_and_totals = get_column_total_values(data)

for col_name, total in sorted(column_name_and_totals.items(), key=lambda kv: kv[1], reverse=True):
    name = _prepare_arabic_text(str(col_name))
    try:
        val = float(total)
        if val.is_integer():
            print(f"Column Name: {name}, Total Value: {int(val)}")
        else:
            print(f"Column Name: {name}, Total Value: {val:.2f}")
    except Exception:
        print(f"Column Name: {name}, Total Value: {total}")


In [ ]:
# Build a totals table from the previously computed dictionary
totals = get_column_total_values(data)

column_totals_df = (
    pd.Series(totals, name="Total Value")
      .rename_axis("Column Name")
      .reset_index()
      .sort_values("Total Value", ascending=False, ignore_index=True)
)

display(column_totals_df)


In [ ]:
from typing import Tuple, Dict, Any
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.utils.multiclass import unique_labels
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    precision_recall_fscore_support,
    confusion_matrix
)

def train_and_evaluate_model(
    X: pd.DataFrame,
    y: pd.Series,
    test_size: float = 0.3,
    random_state: int = 42,
    n_estimators: int = 300,
    max_depth: int | None = None
) -> Tuple[RandomForestClassifier, float, Dict[str, Any]]:
    """
    Train a RandomForest classifier and evaluate it.
    Handles edge cases such as single-class targets and small datasets.
    """
    # Basic validations
    if len(X) != len(y):
        raise ValueError("X and y must have the same number of rows.")
    if len(X) < 3:
        raise ValueError("Dataset too small to split.")
    y = y.astype(int)

    # Stratify if both classes exist
    classes = np.unique(y)
    stratify_arg = y if len(classes) > 1 else None

    X_train, X_test, y_train, y_test = train_test_split(
        X, y,
        test_size=test_size,
        random_state=random_state,
        stratify=stratify_arg
    )

    # If single-class in train, force a trivial classifier
    if len(np.unique(y_train)) < 2:
        majority = int(np.bincount(y).argmax())
        class TrivialRF:
            def predict(self, Z): return np.full(len(Z), majority, dtype=int)
            feature_importances_ = np.zeros(X.shape[1], dtype=float)
        model = TrivialRF()
        y_pred = model.predict(X_test)
        acc = float((y_pred == y_test).mean())
        report = classification_report(y_test, y_pred, target_names=['0', '1'], zero_division=0, output_dict=True)
        metrics = {
            "accuracy": acc,
            "report": report,
            "confusion_matrix": confusion_matrix(y_test, y_pred).tolist(),
            "feature_importances_top5": []
        }
        print(f"Accuracy: {acc:.4f}")
        print(classification_report(y_test, y_pred, target_names=['0','1'], zero_division=0))
        return model, acc, metrics

    # Train RF
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=random_state,
        class_weight="balanced_subsample",
        n_jobs=-1
    )
    model.fit(X_train, y_train)

    # Predict and evaluate
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    report_txt = classification_report(y_test, y_pred, target_names=['مرفوض (0)', 'مقبول (1)'], zero_division=0)
    report = classification_report(y_test, y_pred, zero_division=0, output_dict=True)
    prfs = precision_recall_fscore_support(y_test, y_pred, zero_division=0)
    cm = confusion_matrix(y_test, y_pred)

    # Top 5 important features
    importances = pd.Series(model.feature_importances_, index=X.columns)
    top5 = importances.nlargest(5)

    print(f"Training size: {len(X_train)} | Test size: {len(X_test)}")
    print("Model: RandomForestClassifier")
    print(f"Accuracy: {acc:.4f}")
    print("\nClassification Report:\n", report_txt)
    print("\nTop 5 Feature Importances:")
    print(top5.to_string())

    metrics = {
        "accuracy": acc,
        "report": report,
        "precision_recall_fscore_support": {
            "precision": prfs[0].tolist(),
            "recall": prfs[1].tolist(),
            "f1": prfs[2].tolist(),
            "support": prfs[3].tolist(),
        },
        "confusion_matrix": cm.tolist(),
        "feature_importances_top5": top5.to_dict()
    }
    return model, acc, metrics


In [ ]:
from typing import Tuple, Dict, Any

def predict_single_row(model, raw_row_data: Dict[str, Any], X_train_cols: list) -> Tuple[int, np.ndarray]:
    """
    Preprocess a single raw row, align to training features, and predict.
    Mirrors the steps in preprocess_data() to keep feature engineering consistent.
    Returns (predicted_label, probabilities_array).
    """
    # 1) Build DataFrame
    df = pd.DataFrame([raw_row_data]).copy()

    # 2) Normalize common alternative column names to canonical ones used in training
    rename_map = {
        'اخر معاد لاستلام الاستفسارات': 'اخر موعد لاستلام الاستفسارات',
        'قيمة وثائق المنافسة': 'قيمه وثائق المنافسه',
        'قيمة وثائق المنافسه': 'قيمه وثائق المنافسه',
        'URL': 'url',
        'الرابط': 'url'
    }
    present_map = {k: v for k, v in rename_map.items() if k in df.columns}
    if present_map:
        df = df.rename(columns=present_map)

    # 3) Drop columns not used as features
    drop_cols = ['url', 'اخر موعد لاستلام الاستفسارات', 'الرقم المرجعي', 'حالة القبول']
    df = df.drop(columns=[c for c in drop_cols if c in df.columns], errors='ignore')

    # 4) Clean numeric value column
    val_col = 'قيمه وثائق المنافسه'
    if val_col in df.columns:
        ser = (
            df[val_col].astype(str)
            .str.replace(r'مجانا|مجان', '0', flags=re.IGNORECASE, regex=True)
            .str.replace(r'[^\d\.\-]', '', regex=True)
            .str.strip()
        )
        df[val_col] = pd.to_numeric(ser, errors='coerce').fillna(0.0)
    else:
        df[val_col] = 0.0

    # 5) Dates -> engineered features
    date_cols = ['اخر موعد لتقديم العرض', 'تاريخ النشر']
    for c in date_cols:
        if c in df.columns:
            df[c] = pd.to_datetime(df[c], errors='coerce')
        else:
            df[c] = pd.NaT

    df['ايام_متبقية_للعرض'] = (df['اخر موعد لتقديم العرض'] - df['تاريخ النشر']).dt.days
    df['ايام_متبقية_للعرض'] = df['ايام_متبقية_للعرض'].fillna(0).astype(int)

    df['مجانية_المنافسة'] = (df[val_col] == 0).astype(int)

    df = df.drop(columns=[c for c in date_cols if c in df.columns], errors='ignore')

    # 6) Categorical one-hot encoding
    cat_cols = ['النشاط الاساسي', 'العنوان', 'وصف']
    for c in cat_cols:
        if c not in df.columns:
            df[c] = ""
    df[cat_cols] = df[cat_cols].fillna("").astype(str)

    df = pd.get_dummies(df, columns=cat_cols, drop_first=True)

    # 7) Align to training feature columns
    df_aligned = df.reindex(columns=X_train_cols, fill_value=0)

    # 8) Predict
    pred = int(model.predict(df_aligned)[0])
    if hasattr(model, "predict_proba"):
        proba = model.predict_proba(df_aligned)[0]
    else:
        # Fallback: pseudo-proba for models without predict_proba
        proba = np.array([1.0 - float(pred), float(pred)], dtype=float)

    return pred, proba


In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default

creds, _ = default()
gc = gspread.authorize(creds)
ws = gc.open_by_key(SHEET_ID).sheet1


In [ ]:
# --- Read target sheet (OAuth path, no credentials.json) ---
# assumes: gc is already defined from auth.authenticate_user()
import pandas as pd

TARGET_SHEET_ID = "15F9xOSLfXDuIQBDIhbW2g6ZFWlkTRmEX2507KCSP5UI"
TARGET_TAB = "Sheet1"  # change if different

wb2 = gc.open_by_key(TARGET_SHEET_ID)
tabs = [w.title for w in wb2.worksheets()]
ws2 = wb2.worksheet(TARGET_TAB) if TARGET_TAB in tabs else wb2.sheet1

rows = ws2.get_all_records()         # row 1 = headers
df_target = pd.DataFrame(rows)

# normalize headers to canonical names used downstream
rename_map = {
    "URL": "url",
    "الرابط": "url",
    "اخر معاد لاستلام الاستفسارات": "اخر موعد لاستلام الاستفسارات",
    "قيمة وثائق المنافسة": "قيمه وثائق المنافسه",
    "قيمة وثائق المنافسه": "قيمه وثائق المنافسه",
}
present = {k: v for k, v in rename_map.items() if k in df_target.columns}
if present:
    df_target = df_target.rename(columns=present)

# keep only needed columns if present
needed = ["الرقم المرجعي", "حالة القبول"]
missing = [c for c in needed if c not in df_target.columns]
if missing:
    print(f"Warning: missing columns in target sheet: {missing}")
else:
    for c in needed:
        df_target[c] = df_target[c].astype(str).str.strip()

print(f"Loaded target sheet rows: {len(df_target)} | cols: {len(df_target.columns)}")
display(df_target.head())


In [ ]:
# === Define, normalize, and sanity-check join key in one shot ===
import pandas as pd

# 1) define df_main safely
if 'data' in globals() and isinstance(data, pd.DataFrame):
    df_main = data.copy()
elif 'df' in globals() and isinstance(df, pd.DataFrame):
    df_main = df.copy()
else:
    raise RuntimeError("df_main source not found: expected a DataFrame named 'data' or 'df'.")

# 2) ensure df_target exists
if 'df_target' not in globals() or not isinstance(df_target, pd.DataFrame):
    raise RuntimeError("df_target is missing. Load your target sheet into a DataFrame named 'df_target' first.")

# 3) normalize the join key
KEY_COL = 'الرقم المرجعي'

def _normalize_key_col(df: pd.DataFrame, key=KEY_COL) -> pd.DataFrame:
    if key not in df.columns:
        raise KeyError(f"Missing join key column: {key}")
    s = df[key].astype(str)
    s = (s.str.replace('\u200f','', regex=False)   # remove RTL marks
           .str.replace(r'\s+','', regex=True)     # remove spaces
           .str.replace('٬','', regex=False)       # Arabic thousands sep
           .str.replace('٫','.', regex=False)      # Arabic decimal
           .str.replace(r'[^\d]','', regex=True)   # keep digits only
           .str.strip())
    df[key] = s
    return df

df_main   = _normalize_key_col(df_main)
df_target = _normalize_key_col(df_target)

# 4) quick diagnostics (optional)
km = set(df_main[KEY_COL].dropna())
kt = set(df_target[KEY_COL].dropna())
print("df_main rows:", len(df_main), "| unique keys:", len(km))
print("df_target rows:", len(df_target), "| unique keys:", len(kt))
print("intersection size:", len(km & kt))


In [ ]:
# Normalize join key in both dataframes to string
def _normalize_key_col(df, key='الرقم المرجعي'):
    if key not in df.columns:
        raise KeyError(f"Missing join key column: {key}")
    s = df[key].astype(str)
    s = (s.str.replace('\u200f', '', regex=False)       # remove RTL marks if any
           .str.replace(r'\s+', '', regex=True)         # remove spaces
           .str.replace('٬', '', regex=False)           # thousands sep
           .str.replace('٫', '.', regex=False)          # decimal sep
           .str.replace(r'[^\d]', '', regex=True)       # keep digits only (avoids float precision loss)
           .str.strip())
    df[key] = s
    return df

df_main = _normalize_key_col(df_main.copy(), 'الرقم المرجعي')
df_target = _normalize_key_col(df_target.copy(), 'الرقم المرجعي')


In [ ]:
# === Safe join diagnose + fallback + optional training ===
import pandas as pd
import numpy as np

# 0) Preconditions
if 'data' in globals() and isinstance(data, pd.DataFrame):
    df_main = data.copy()
elif 'df' in globals() and isinstance(df, pd.DataFrame):
    df_main = df.copy()
else:
    raise RuntimeError("Main DataFrame not found. Expected 'data' or 'df'.")

assert 'df_target' in globals() and isinstance(df_target, pd.DataFrame), "df_target is missing."

KEY_COL = 'الرقم المرجعي'

def _norm(s: pd.Series) -> pd.Series:
    return (s.astype(str)
             .str.replace('\u200f','', regex=False)
             .str.replace(r'\s+','', regex=True)
             .str.replace('٬','', regex=False)
             .str.replace('٫','.', regex=False)
             .str.replace(r'[^\d]','', regex=True)
             .str.strip())

# 1) Normalize keys
if KEY_COL not in df_main.columns: raise KeyError(f"Missing '{KEY_COL}' in df_main")
if KEY_COL not in df_target.columns: raise KeyError(f"Missing '{KEY_COL}' in df_target")
dfm = df_main.copy(); dfm[KEY_COL] = _norm(dfm[KEY_COL])
dft = df_target.copy(); dft[KEY_COL] = _norm(dft[KEY_COL])

# 2) Direct intersection
km = set(dfm[KEY_COL].dropna())
kt = set(dft[KEY_COL].dropna())
inter = km & kt
print(f"[Direct] main unique: {len(km)} | target unique: {len(kt)} | intersection: {len(inter)}")

# 3) If no intersection, try last-9-digits join (fallback)
use_tail_join = False
if len(inter) == 0:
    dfm['_k9'] = dfm[KEY_COL].str[-9:]
    dft['_k9'] = dft[KEY_COL].str[-9:]
    inter9 = set(dfm['_k9']) & set(dft['_k9'])
    print(f"[Tail-9] intersection: {len(inter9)}")
    if len(inter9) > 0:
        use_tail_join = True

# 4) Build aligned labeled set
if use_tail_join:
    df_merged = pd.merge(
        dfm, dft[['_k9','حالة القبول']], on='_k9', how='inner'
    )
    # reattach canonical key if needed
    if KEY_COL not in df_merged.columns and KEY_COL in dfm.columns:
        pass
else:
    df_merged = pd.merge(
        dfm, dft[[KEY_COL,'حالة القبول']], on=KEY_COL, how='inner'
    )

print(f"Merged labeled rows: {len(df_merged)}")

# 5) If too small, skip training gracefully
if len(df_merged) < 3:
    print("Not enough labeled rows to train a model. Skipping ML training.")
    trained_model, accuracy, metrics = None, None, {}
else:
    # Split back into df_main-like and df_target-like for your preprocess
    cols_target = ['حالة القبول']
    df_main_for_ml = df_merged.drop(columns=[c for c in cols_target if c in df_merged.columns]).copy()
    df_target_for_ml = df_merged[[KEY_COL, 'حالة القبول']].copy() if KEY_COL in df_merged.columns else df_merged[['حالة القبول']].copy()

    # Ensure required columns exist
    if KEY_COL not in df_main_for_ml.columns and KEY_COL in dfm.columns:
        df_main_for_ml[KEY_COL] = dfm[KEY_COL]

    # Call your existing pipeline
    X_features, y_target = preprocess_data(df_main_for_ml, df_target_for_ml)
    if len(X_features) < 3:
        print("Not enough rows after preprocessing. Skipping ML training.")
        trained_model, accuracy, metrics = None, None, {}
    else:
        trained_model, accuracy, metrics = train_and_evaluate_model(X_features, y_target)
        print(f"Accuracy: {accuracy:.4f}")
        if isinstance(metrics, dict) and 'report' in metrics:
            print("Classes in report:", list(metrics['report'].keys()))


In [ ]:
# One-shot: normalize key, preprocess, guard small data, train if possible
import pandas as pd

# 1) define main df safely
df_main = data.copy() if 'data' in globals() else df.copy()

# 2) ensure df_target exists
assert 'df_target' in globals(), "df_target is missing."

KEY_COL = 'الرقم المرجعي'

def _norm(s: pd.Series) -> pd.Series:
    return (s.astype(str)
             .str.replace('\u200f','', regex=False)
             .str.replace(r'\s+','', regex=True)
             .str.replace('٬','', regex=False)
             .str.replace('٫','.', regex=False)
             .str.replace(r'[^\d]','', regex=True)
             .str.strip())

# 3) normalize join key types
if KEY_COL not in df_main.columns: raise KeyError(f"Missing '{KEY_COL}' in df_main")
if KEY_COL not in df_target.columns: raise KeyError(f"Missing '{KEY_COL}' in df_target")
df_main[KEY_COL]   = _norm(df_main[KEY_COL])
df_target[KEY_COL] = _norm(df_target[KEY_COL])

# 4) quick intersection check to avoid empty merge
inter = set(df_main[KEY_COL]) & set(df_target[KEY_COL])
if len(inter) < 3:
    print(f"Not enough labeled intersection ({len(inter)}). Skipping ML.")
    trained_model, accuracy, metrics = None, None, {}
    X_features, y_target, X_train_cols = pd.DataFrame(), pd.Series(dtype=int), []
else:
    X_features, y_target = preprocess_data(df_main, df_target)
    if len(X_features) < 3:
        print("Not enough rows after preprocessing. Skipping ML.")
        trained_model, accuracy, metrics = None, None, {}
        X_train_cols = []
    else:
        trained_model, accuracy, metrics = train_and_evaluate_model(X_features, y_target)
        X_train_cols = X_features.columns.tolist()
        print(f"Accuracy: {accuracy:.4f}")
        if isinstance(metrics, dict) and "report" in metrics:
            print("Classes in report:", list(metrics["report"].keys()))


In [ ]:
# --- Locate a valid row by reference id (interactive fallback) ---

import pandas as pd
from difflib import get_close_matches

# 1) choose main df
base_df = data if 'data' in globals() else df_main

REF_COL = 'الرقم المرجعي'
assert REF_COL in base_df.columns, f"Missing column: {REF_COL}"

# 2) normalize helper
def norm_digits(s):
    s = str(s)
    for a,b in [('‏',''), (' ','')]: s = s.replace(a,b)
    s = s.replace('٬','').replace('٫','.')
    s = ''.join(ch for ch in s if ch.isdigit())
    return s.strip()

dfN = base_df.copy()
dfN['_ref_norm'] = dfN[REF_COL].astype(str).map(norm_digits)

# 3) candidate list
cands = dfN['_ref_norm'].dropna().unique().tolist()
print(f"Available refs ({len(cands)}):", cands[:10], ("..." if len(cands)>10 else ""))

# 4) put your desired id here; if empty, we'll pick the first
TARGET_ID = ""  # e.g. "251039012564"
TARGET_ID = norm_digits(TARGET_ID) if TARGET_ID else ""

row = None
reason = None

if TARGET_ID and TARGET_ID in cands:
    row = dfN[dfN['_ref_norm'] == TARGET_ID].iloc[0]
    reason = "exact"
elif TARGET_ID:
    # try close matches (string similarity)
    close = get_close_matches(TARGET_ID, cands, n=1, cutoff=0.8)
    if close:
        row = dfN[dfN['_ref_norm'] == close[0]].iloc[0]
        reason = f"closest:{close[0]}"
    else:
        # fallback to first candidate if nothing close
        row = dfN.iloc[0]
        reason = "fallback:first-row"
else:
    # user didn't specify; just take first row
    row = dfN.iloc[0]
    reason = "default:first-row"

print(f"Selected row reason: {reason}")
print("Selected ref:", row['_ref_norm'])

raw_value = row.drop(labels=['_ref_norm']).to_dict()

# 5) predict only if model trained; otherwise just show the row
can_predict = (
    'trained_model' in globals() and trained_model is not None and
    'X_train_cols' in globals() and isinstance(X_train_cols, list) and len(X_train_cols) > 0
)

if can_predict:
    pred_label, proba = predict_single_row(trained_model, raw_value, X_train_cols)
    label_text = "مقبول" if pred_label == 1 else "مرفوض"
    confidence = float(proba[pred_label])
    print("\n--- Prediction ---")
    print(f"Label: {label_text} | Confidence: {confidence:.2%}")
else:
    print("\nModel not trained or X_train_cols missing. Showing selected row only:")

import pprint; pprint.pprint(raw_value)


In [ ]:
# Build analyzer and load data
analyzer = TenderDataAnalyzer()
analyzer.load_data_from_text(data)  # أو df_main لو ده اسمك

# Generate PDF report
REPORT_FILE = f"Tender_Report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.pdf"
generated_pdf_path = create_full_pdf_report(analyzer, REPORT_FILE)
print("PDF ready at:", generated_pdf_path)

# Send report by email (ضبطي الإيميل/كلمة مرور التطبيق في الإعدادات)
RECIPIENT_EMAIL = "basma66yy@gmail.com"
send_email_with_pdf(generated_pdf_path, RECIPIENT_EMAIL)
